# Enumeração Exaustiva

Este código é uma implementação do método de enumeração exaustiva para a estimação dos índices anuais de confiabilidade:

- LOLP

- LOLE

- EPNS

- EENS

- EIR

### Importar Bibliotecas

In [35]:
import numpy as np
from scipy.stats import binom
import pandas as pd
from itertools import combinations, product
import openpyxl

carga = 2850

### Definir classe Agrupamento

Classe que modela o agrupamento de unidades da mesma usina e calcula a probabilidade de falha.

In [36]:
class Agrupamento:
    def __init__(self, pot, qtd, prob):
        self.pot = pot
        self.qtd = qtd
        self.prob = prob
        self.geraTabela()

        self.pot_max = self.pot * self.qtd

    def geraTabela(self):
        self.resultados = []
        self.iterado = []
        for i in range(int(self.qtd) + 1):
            self.resultados.append((i * self.pot, binom.pmf(i, int(self.qtd), self.prob)))
            self.iterado.append(0)
        self.usina = pd.DataFrame(self.resultados,columns=['Pot','Prob'])
        self.usina['Iter'] = self.iterado
        self.usina['IterAg'] = np.nan


### Definir função Calcula_COPT 

Função que gera a tabela COPT e calcula os índices de confiabilidade do sistema para cada valor de carga.
Essa função recebe como parâmetros:
- data: dados das usinas;
- carga: valor de carga, a ser variado de acordo com os valores anuais da curva de carga;
- falhas_simultaneas: critério de número de falhas simultâneas a ser analisadas.


In [37]:
def Calcula_COPT(data, carga, falhas_simultaneas):
    agrupamento = []
    agrpot = []
    agrprob = []
    resul = []
    usinas = []
    geracao_total = 0
    for i in range(len(data['Usina'])):
        agrupamento.append(Agrupamento(data.loc[i]['Pot. Ativa Max'], data.loc[i]['Unidades'], data.loc[i]['FOR']))
        agrpot.append(list(zip(*agrupamento[i].resultados))[0])
        agrprob.append(list(zip(*agrupamento[i].resultados))[1])
        usinas.append(agrupamento[i].usina)
        geracao_total += data.loc[i]['Unidades']*data.loc[i]['Pot. Ativa Max']
    
    potencia_para_falta = geracao_total - carga
    
    lista = []
    for i in range(len(usinas)):
        for x in range(len(usinas[i])):
            lista.append((i,x))

    # se 'falhas_simultaneas' for um numero inteiro, usar apenas esse numero de falhas
    if isinstance(falhas_simultaneas, int):
        resultado_pot_teste = []
        resultado_prob_teste = []
        
        n_usinas = len(usinas)
        prod_prob = list(product(*agrprob))

        for it, comb in enumerate(product(*agrpot)):

            # o numero de zeros é o numero de usinas sem falha; 
            # caso haja mais usinas sem falha, armazenar a potencia e a probabilidade
            if comb.count(0) >= n_usinas - falhas_simultaneas:
                resultado_pot_teste.append(sum(comb))
                resultado_prob_teste.append(np.prod(prod_prob[it]))

            # caso haja mais usinas com falha, não armazenar. continua p/ proxima combinação


    # se 'falhas_simultaneas' não for inteiro, fazer com todas as possibilidades
    else:
        resultado_pot_teste = []
        for i in product(*agrpot):
            resultado_pot_teste.append(sum(i))
        resultado_prob_teste = []
        for i in product(*agrprob):
            resultado_prob_teste.append(np.prod(i))



    copt = pd.DataFrame(resultado_pot_teste, columns=['Pot indisponivel'])
    copt['Probabilidade'] = resultado_prob_teste
    copt = copt.groupby(by = ['Pot indisponivel'], as_index = False).sum()
    
    perda_de_carga = []
    
    for i in range(len(copt)):
        if copt.loc[i]['Pot indisponivel'] <=  potencia_para_falta:
            perda_de_carga.append(0)
        else:
            perda_de_carga.append(abs(copt.loc[i]['Pot indisponivel'] - potencia_para_falta))
    
    
    copt['Perda de Carga'] = perda_de_carga
    copt['x * p'] = copt['Perda de Carga'] * copt['Probabilidade']
    
    EPNS = sum(copt['x * p'])
    LOLP = sum(copt[copt['Perda de Carga'] > 0]['Probabilidade'])
    LOLE = LOLP * 8760
    EENS = EPNS * 8760
    resul_copt = [EPNS, EENS, LOLP, LOLE]
    return resul_copt,copt



def pu(x):
    return x/100

## Programa Principal

### Leitura de Dados

A leitura dos dados das usinas é feita a partir do arquivo "Gerac.xlsx".

In [38]:
# data = pd.read_excel('Gerac.xlsx')
# data['FOR'] = data['FOR'].apply(pu)


# Agrupar usinas que tenham as mesmas potências e FORs

data = pd.read_excel('Gerac.xlsx')

data2 = data.groupby(by = ['Pot. Ativa Max','FOR'], as_index=False, sort=False,group_keys=False)['Unidades'].sum().reset_index()
data2['Usina'] = data['Usina']
data = data2

data['FOR'] = data['FOR'].apply(pu)

# data

A leitura dos dados da curva de carga é feita a partir do arquivo "dataframe_curva_de_carga.xlsx". 

Este arquivo é gerado a partir do arquivo "ler_curva.ipynb", que, para isso, lê os dados do arquivo "curva de carga.xlsx".

In [39]:
dados_carga = pd.read_excel('dataframe_curva_de_carga.xlsx')

# dados_carga

### Definição do Critério de Falhas Simultâneas Adotado

Caso se deseje simular o critério N-2, alterar o valor da variável "falhas_simultaneas" para 2.

Para simular todas as combinações possíveis, alterar o valor da variável "falhas_simultaneas" para "all".

In [40]:
# Número de falhas simultâneas ('all' para utilizar todas as combinações possíveis)
FALHAS_SIMULTANEAS = 'all'

### Cálculo dos Índices de Confiabilidade

Esta é a etapa principal do programa, na qual, para cada valor de carga, são calculados os índices de confiabilidade do sistema.

In [41]:
EPNS = []
EENS = []
LOLP = []
LOLE = []
valores_de_carga = dados_carga['Valor de Carga'].values

for n,i in enumerate(valores_de_carga):
    result, copt = Calcula_COPT(data, carga*i/100, FALHAS_SIMULTANEAS)
    EPNS.append(result[0]*dados_carga['Probabilidade'][n])
    EENS.append(result[1]*dados_carga['Probabilidade'][n])
    LOLP.append(result[2]*dados_carga['Probabilidade'][n])
    LOLE.append(result[3]*dados_carga['Probabilidade'][n])


In [42]:
# calcular e guardar COPT do caso com 100% de carga
result_100, copt_100 = Calcula_COPT(data, carga*100/100, FALHAS_SIMULTANEAS)

Obtenção dos valores finais dos índices:


In [43]:
LOLP = sum(LOLP)
EPNS = sum(EPNS)
LOLE = sum(LOLE)
EENS = sum(EENS)

E_sob_curva = sum(dados_carga['Valor de Carga'] * dados_carga['Probabilidade'] * 8760) * carga

EIR = 1 - EENS/E_sob_curva

### Exibição dos Resultados

##### Índices de confiabilidade 

Exibir valores dos índices

In [44]:
# criar um dataframe com os resultados, no qual os indices são os resultados e as colunas são os valores

resultado = pd.DataFrame([LOLP, EPNS, LOLE, EENS, EIR], index=['LOLP', 'EPNS (MW)', 'LOLE (h/ano)', 'EENS (MWh/ano)', 'EIR'], columns=['Valor'])

# alinhar à esquerda
resultado = resultado.style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

resultado

,Valor
LOLP,0.023381
EPNS (MW),3.636702
LOLE (h/ano),204.820854
EENS (MWh/ano),31857.507169
EIR,0.999985


Escrever índices na planilha "EnumExaust" do arquivo "Resultados.xlsx"

In [45]:
# apagar a planilha "EnumExaust" do arquivo "Resultados.xlsx"
wb = openpyxl.load_workbook('Resultados.xlsx')
if 'EnumExaust' in wb.sheetnames:
    wb.remove(wb['EnumExaust'])
    wb.save('Resultados.xlsx')
wb.close()

# escrever o resultado na planilha "EnumExaust" do arquivo "Resultados.xlsx", sem apagar o que já está escrito nas outras
writer = pd.ExcelWriter('Resultados.xlsx', engine='openpyxl', mode='a')
resultado.to_excel(writer, sheet_name='EnumExaust')
writer.close()

##### COPT para o caso de 100% de carga

Exibir COPT: dataframe (recorte)

In [47]:
copt_100

,Pot indisponivel,Probabilidade,Perda de Carga,x * p
0,0.0,2.363951e-01,0.0,0.000000e+00
1,12.0,2.412195e-02,0.0,0.000000e+00
2,20.0,1.050645e-01,0.0,0.000000e+00
3,24.0,9.845694e-04,0.0,0.000000e+00
4,32.0,1.072087e-02,0.0,0.000000e+00
...,...,...,...,...
3175,3373.0,1.065420e-44,2818.0,3.002354e-41
3176,3381.0,2.900311e-44,2826.0,8.196279e-41
3177,3385.0,4.348654e-47,2830.0,1.230669e-43
3178,3393.0,2.959501e-46,2838.0,8.399064e-43


Exibir COPT: texto (completa)

In [46]:
# Exibir COPT para o caso de 100% de carga
print(copt_100.to_string())

      Pot indisponivel  Probabilidade  Perda de Carga         x * p
0                  0.0   2.363951e-01             0.0  0.000000e+00
1                 12.0   2.412195e-02             0.0  0.000000e+00
2                 20.0   1.050645e-01             0.0  0.000000e+00
3                 24.0   9.845694e-04             0.0  0.000000e+00
4                 32.0   1.072087e-02             0.0  0.000000e+00
5                 36.0   2.009325e-05             0.0  0.000000e+00
6                 40.0   1.751075e-02             0.0  0.000000e+00
7                 44.0   4.375864e-04             0.0  0.000000e+00
8                 48.0   2.050332e-07             0.0  0.000000e+00
9                 50.0   1.432698e-02             0.0  0.000000e+00
10                52.0   1.786811e-03             0.0  0.000000e+00
11                56.0   8.930335e-06             0.0  0.000000e+00
12                60.0   1.297093e-03             0.0  0.000000e+00
13                62.0   1.461936e-03           